In [0]:
CATALOGO = "v_credit"
SCHEMA_ORIGEM = "gold"
SCHEMA_DESTINO = "curated"
NOME_VIEW = "vw_custo_oportunidade"

full_view_name = f"{CATALOGO}.{SCHEMA_DESTINO}.{NOME_VIEW}"

In [0]:
query = f"""
CREATE OR REPLACE VIEW {full_view_name} AS
SELECT
    -- Dimensões
    canal.nm_canal,
    motivo.ds_categoria,
    motivo.ds_motivo,
    motivo.ds_potencial_automacao,
    
    -- Métricas Financeiras
    COUNT(fato.pk_fato_atendimento) AS qtd_chamados,
    SUM(fato.val_custo) AS custo_total_reais,
    AVG(fato.val_custo) AS custo_medio_reais,
    
    -- Métrica de Eficiência
    AVG(fato.val_tempo_atendimento) AS tma_medio_segundos

FROM {CATALOGO}.{SCHEMA_ORIGEM}.ft_atendimento_geral AS fato
JOIN {CATALOGO}.{SCHEMA_ORIGEM}.dm_motivo AS motivo ON fato.cd_motivo = motivo.cd_motivo
JOIN {CATALOGO}.{SCHEMA_ORIGEM}.dm_canal AS canal ON fato.cd_canal = canal.cd_canal

-- Filtro: Focamos apenas onde HÁ CUSTO (ignorando interações gratuitas se houver)
WHERE fato.val_custo > 0
GROUP BY 1, 2, 3, 4;
"""

spark.sql(query)
print(f"✅ View {full_view_name} criada/atualizada com sucesso!")